In [5]:
# level55_execution_sim.py
# Level-55: Execution Simulator – TWAP/VWAP with participation caps & impact model.
# Data: yfinance intraday (1m ≤ 7d; else 5m). Outputs CSV + JSON.
#
# Usage (examples):
#   python level55_execution_sim.py --symbol SPY --side buy --qty 50000 --minutes 120 --strategy twap
#   python level55_execution_sim.py --symbol AAPL --side sell --qty 25000 --minutes 90 --strategy vwap --participation 0.12
#   python level55_execution_sim.py --symbol BTC-USD --side buy --qty 5 --minutes 60 --interval 1m --spread-bps 1.5
#
# Notes:
#   - For crypto, use symbols like "BTC-USD" (yfinance supports them).
#   - 1-minute data is available up to 7 days. For longer runs, pick interval 5m.
#   - All prices in the downloaded timeframe's timezone (use UTC index for consistency).

import os
import json
import argparse
from dataclasses import dataclass, asdict

import numpy as np
import pandas as pd
import yfinance as yf


# ----------------------------- Config -----------------------------
@dataclass
class Config:
    symbol: str = "SPY"
    side: str = "buy"                 # buy or sell
    qty: float = 50000                # parent quantity (shares/units)
    minutes: int = 120                # total schedule horizon (minutes)
    strategy: str = "twap"            # twap or vwap
    interval: str = "1m"              # 1m or 5m
    lookback_days: int = 5            # how many recent days to pull
    start_offset_min: int = 0         # start after N mins from first bar
    participation: float = 0.10       # max participation per bar (0..1)
    spread_bps: float = 2.0           # assumed quoted spread (round-trip bps of mid); half-spread applied per fill
    k_temp: float = 0.30              # temp impact coefficient
    alpha: float = 0.60               # temp impact elasticity on (child/bar_vol)
    k_perm: float = 0.02              # permanent impact coefficient vs cum_exec/ADV
    vol_span: int = 20                # EWMA span for volatility estimate
    vol_floor_bps: float = 5.0        # minimum σ per bar in bps to avoid zero impact
    out_csv: str = "level55_exec_slices.csv"
    out_json: str = "level55_exec_summary.json"
    seed: int = 42                    # for any stochastic extensions (kept deterministic here)


# ----------------------------- Helpers -----------------------------
def _to_float(x) -> float:
    """
    Safe float conversion that avoids FutureWarning on single-element Series.
    """
    if isinstance(x, pd.Series):
        # single element Series → use iloc[0] as pandas recommends
        if len(x) == 0:
            return float("nan")
        return float(x.iloc[0])
    return float(x)


# ----------------------------- Data Loader -----------------------------
def load_intraday(cfg: Config) -> pd.DataFrame:
    # Enforce 1m only when lookback ≤ 7 days (yfinance limit). Else switch to 5m.
    interval = cfg.interval
    if interval == "1m" and cfg.lookback_days > 7:
        interval = "5m"
    period = f"{cfg.lookback_days}d"
    df = yf.download(cfg.symbol, period=period, interval=interval, auto_adjust=True, progress=False)
    if df.empty:
        raise RuntimeError("No intraday data returned (symbol/period/interval mismatch?).")
    # Keep essential columns; ensure datetime index tz-naive (we'll treat as UTC-like for math)
    if isinstance(df.index, pd.DatetimeIndex):
        try:
            df = df.tz_convert("UTC")
        except Exception:
            pass
        try:
            df = df.tz_localize(None)
        except Exception:
            pass
    df = df.rename(columns=str.title)
    need = ["Open", "High", "Low", "Close", "Volume"]
    for c in need:
        if c not in df.columns:
            raise RuntimeError(f"Missing column in data: {c}")
    df = df[need].dropna()
    return df


# ----------------------------- Volume Curve (VWAP) -----------------------------
def volume_curve_weights(bars: pd.DataFrame) -> pd.Series:
    """
    Estimate an intraday volume curve using a rolling median by minute-of-day.
    Fallback to simple proportional to bar volume if little history.
    """
    if "Volume" not in bars:
        raise ValueError("bars need Volume")
    i = bars.index
    if not isinstance(i, pd.DatetimeIndex):
        raise ValueError("bars index must be DatetimeIndex")
    # minute-of-day key
    mod = i.hour * 60 + i.minute
    df = bars.copy()
    df["mod"] = mod
    # Rolling median per minute-of-day (requires multiple days to stabilize)
    curve = df.groupby("mod")["Volume"].median()
    curve = curve.reindex(mod, method="nearest")  # map back
    curve = curve.fillna(curve.median())
    w = curve / curve.sum()
    w.index = bars.index
    return w


# ----------------------------- Volatility per bar -----------------------------
def bar_volatility_ewma(bars: pd.DataFrame, span: int, floor_bps: float) -> pd.Series:
    mid = (bars["High"] + bars["Low"]) / 2.0
    r = np.log(mid).diff().fillna(0.0)
    ew = r.ewm(span=span, adjust=False).std().fillna(0.0)
    # convert to bps per bar; floor
    sigma_bps = (ew * 1e4).clip(lower=floor_bps)
    return sigma_bps


# ----------------------------- Schedule Builder -----------------------------
def build_schedule(cfg: Config, bars: pd.DataFrame) -> pd.DataFrame:
    """
    Build per-bar target quantities for TWAP/VWAP over a horizon of cfg.minutes.
    """
    # Choose the starting index
    bars = bars.copy()
    start_idx = cfg.start_offset_min
    # Determine how many rows correspond to horizon, given actual interval
    dt = (bars.index[1] - bars.index[0])
    bar_minutes = int(round(dt.total_seconds() / 60.0))
    if bar_minutes <= 0:
        bar_minutes = 1
    n_slices = max(1, cfg.minutes // bar_minutes)

    window = bars.iloc[start_idx:start_idx + n_slices].copy()
    if window.empty or len(window) < 1:
        raise RuntimeError("Chosen minutes/start_offset exceed available intraday window.")

    # weights
    if cfg.strategy.lower() == "twap":
        w = pd.Series(1.0, index=window.index)
        w = w / w.sum()
    elif cfg.strategy.lower() == "vwap":
        w = volume_curve_weights(window)
    else:
        raise ValueError("strategy must be 'twap' or 'vwap'")

    window["target_qty"] = cfg.qty * w

    # auxiliary features
    window["Mid"] = (window["High"] + window["Low"]) / 2.0
    window["HalfSpread"] = window["Mid"] * (cfg.spread_bps * 0.0001) / 2.0
    window["Sigma_bps"] = bar_volatility_ewma(window, cfg.vol_span, cfg.vol_floor_bps)  # bps per bar
    return window


# ----------------------------- ADV estimate -----------------------------
def estimate_ADV(bars: pd.DataFrame, n_days: int = 20) -> float:
    """
    Simple ADV estimate from last n_days of daily volume using intraday bars.
    """
    # Resample to daily sum of volume
    daily = bars["Volume"].resample("1D").sum()
    adv_val = daily.tail(n_days).mean() if daily.shape[0] else bars["Volume"].mean() * 390
    adv = _to_float(adv_val)
    return max(adv, 1.0)


# ----------------------------- Execution Engine -----------------------------
def simulate_exec(cfg: Config, window: pd.DataFrame, adv_est: float) -> pd.DataFrame:
    """
    Step through each bar. Enforce participation cap, price each child with spread + impact.
    Impact model:
       temp = k_temp * (child / max(1, bar_vol))^alpha * (sigma_bps/1e4) * Mid
       perm = k_perm * (cum_exec / ADV) * Mid
       fill = Mid ± HalfSpread ± temp ± perm   (± based on side)
    """
    side = cfg.side.lower()
    sgn = 1.0 if side == "buy" else -1.0
    if side not in ("buy", "sell"):
        raise ValueError("side must be 'buy' or 'sell'")

    remaining = cfg.qty
    rows = []
    arrival_mid = float(window["Mid"].iloc[0])

    cum_exec = 0.0

    for ts, row in window.iterrows():
        if remaining <= 0:
            # after completion, we still log zero slices to keep shape if desired
            rows.append({
                "time": ts,
                "bar_vol": _to_float(row["Volume"]),
                "target": 0.0,
                "child": 0.0,
                "mid": _to_float(row["Mid"]),
                "half_spread": _to_float(row["HalfSpread"]),
                "sigma_bps": _to_float(row["Sigma_bps"]),
                "temp_impact": 0.0,
                "perm_impact": 0.0,
                "fill_px": np.nan,
                "cum_exec": _to_float(cum_exec),
                "remaining": _to_float(remaining),
            })
            continue

        bar_vol = _to_float(row["Volume"])
        target = _to_float(row["target_qty"])

        # Participation cap
        cap_qty = cfg.participation * bar_vol
        child = min(target, cap_qty, remaining)
        if child <= 0 or bar_vol <= 0:
            rows.append({
                "time": ts,
                "bar_vol": bar_vol,
                "target": target,
                "child": 0.0,
                "mid": _to_float(row["Mid"]),
                "half_spread": _to_float(row["HalfSpread"]),
                "sigma_bps": _to_float(row["Sigma_bps"]),
                "temp_impact": 0.0,
                "perm_impact": 0.0,
                "fill_px": np.nan,
                "cum_exec": _to_float(cum_exec),
                "remaining": _to_float(remaining),
            })
            continue

        # Impact pieces
        ratio = min(1.0, child / max(1.0, bar_vol))
        sigma_bps = _to_float(row["Sigma_bps"])
        mid_px = _to_float(row["Mid"])
        half_spread = _to_float(row["HalfSpread"])

        temp = cfg.k_temp * (ratio ** cfg.alpha) * (sigma_bps * 1e-4) * mid_px
        perm = cfg.k_perm * (cum_exec / max(1.0, adv_est)) * mid_px

        # Fill price (buy => pay +; sell => receive -)
        fill_px = mid_px + sgn * (half_spread + temp + perm)

        # Update execution state
        cum_exec += child
        remaining -= child

        rows.append({
            "time": ts,
            "bar_vol": bar_vol,
            "target": target,
            "child": child,
            "mid": mid_px,
            "half_spread": half_spread,
            "sigma_bps": sigma_bps,
            "temp_impact": temp,
            "perm_impact": perm,
            "fill_px": fill_px,
            "cum_exec": cum_exec,
            "remaining": remaining,
        })

    out = pd.DataFrame(rows).set_index("time")

    # KPIs
    # Implementation shortfall vs arrival mid:
    filled = out["child"] > 0
    if filled.any():
        exec_avg = float(np.average(out.loc[filled, "fill_px"], weights=out.loc[filled, "child"]))
    else:
        exec_avg = arrival_mid

    if side == "buy":
        shortfall = (exec_avg - arrival_mid) / arrival_mid * 1e4  # bps
    else:
        shortfall = (arrival_mid - exec_avg) / arrival_mid * 1e4  # bps

    out.attrs["arrival_mid"] = arrival_mid
    out.attrs["exec_avg"] = exec_avg
    out.attrs["shortfall_bps"] = shortfall
    out.attrs["completed_qty"] = float(out["child"].sum())
    out.attrs["completion_pct"] = 100.0 * out.attrs["completed_qty"] / cfg.qty
    return out


# ----------------------------- I/O -----------------------------
def save_outputs(slices: pd.DataFrame, cfg: Config):
    os.makedirs(os.path.dirname(cfg.out_csv) or ".", exist_ok=True)
    os.makedirs(os.path.dirname(cfg.out_json) or ".", exist_ok=True)

    # CSV
    slices.to_csv(cfg.out_csv, index=True, date_format="%Y-%m-%d %H:%M:%S")

    # JSON summary
    summary = {
        "config": asdict(cfg),
        "arrival_mid": slices.attrs.get("arrival_mid"),
        "exec_avg": slices.attrs.get("exec_avg"),
        "shortfall_bps": slices.attrs.get("shortfall_bps"),
        "completed_qty": slices.attrs.get("completed_qty"),
        "completion_pct": slices.attrs.get("completion_pct"),
        "slices": int(slices.shape[0])
    }
    with open(cfg.out_json, "w") as f:
        json.dump(summary, f, indent=2)

    print(f"[OK] Saved slices → {cfg.out_csv}")
    print(f"[OK] Saved summary → {cfg.out_json}")
    print(f"Arrival mid: {summary['arrival_mid']:.6f} | Exec avg: {summary['exec_avg']:.6f} | Shortfall: {summary['shortfall_bps']:.2f} bps")
    print(f"Completed: {summary['completed_qty']:.2f} ({summary['completion_pct']:.1f}%) across {summary['slices']} slices")


# ----------------------------- CLI -----------------------------
def parse_args() -> Config:
    p = argparse.ArgumentParser(description="Level-55: Execution Simulator (TWAP/VWAP)")
    p.add_argument("--symbol", type=str, default="SPY")
    p.add_argument("--side", type=str, default="buy", choices=["buy", "sell"])
    p.add_argument("--qty", type=float, default=50000)
    p.add_argument("--minutes", type=int, default=120)
    p.add_argument("--strategy", type=str, default="twap", choices=["twap", "vwap"])
    p.add_argument("--interval", type=str, default="1m", choices=["1m", "5m"])
    p.add_argument("--lookback-days", type=int, default=5)
    p.add_argument("--start-offset-min", type=int, default=0)
    p.add_argument("--participation", type=float, default=0.10)
    p.add_argument("--spread-bps", type=float, default=2.0)
    p.add_argument("--k-temp", type=float, default=0.30)
    p.add_argument("--alpha", type=float, default=0.60)
    p.add_argument("--k-perm", type=float, default=0.02)
    p.add_argument("--vol-span", type=int, default=20)
    p.add_argument("--vol-floor-bps", type=float, default=5.0)
    p.add_argument("--csv", type=str, default="level55_exec_slices.csv")
    p.add_argument("--json", type=str, default="level55_exec_summary.json")
    p.add_argument("--seed", type=int, default=42)
    a = p.parse_args()
    return Config(
        symbol=a.symbol, side=a.side, qty=a.qty, minutes=a.minutes, strategy=a.strategy,
        interval=a.interval, lookback_days=a.lookback_days, start_offset_min=a.start_offset_min,
        participation=a.participation, spread_bps=a.spread_bps, k_temp=a.k_temp, alpha=a.alpha,
        k_perm=a.k_perm, vol_span=a.vol_span, vol_floor_bps=a.vol_floor_bps,
        out_csv=a.csv, out_json=a.json, seed=a.seed
    )


# ----------------------------- Main -----------------------------
def main():
    cfg = parse_args()
    np.random.seed(cfg.seed)

    bars = load_intraday(cfg)
    adv = estimate_ADV(bars, n_days=20)

    window = build_schedule(cfg, bars)
    slices = simulate_exec(cfg, window, adv_est=adv)
    save_outputs(slices, cfg)


if __name__ == "__main__":
    # Jupyter/PyCharm cell shim: strip "-f kernel.json" etc.
    import sys
    sys.argv = [sys.argv[0]] + [
        arg for arg in sys.argv[1:]
        if arg != "-f" and not (arg.endswith(".json") and "kernel" in arg)
    ]
    main()


[OK] Saved slices → level55_exec_slices.csv
[OK] Saved summary → level55_exec_summary.json
Arrival mid: 668.559998 | Exec avg: 668.845253 | Shortfall: 4.27 bps
Completed: 50000.00 (100.0%) across 120 slices
